In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
run_name = "vitb_CT-RATE"
checkpoint_name = "training_549999"

test_name = "cross_validation_cls_patch"

results_path = os.path.join("results", run_name, checkpoint_name, test_name)

labels = os.listdir(results_path)
labels

In [ ]:
metrics = ["roc_auc", "pr_auc", "f1"]
cv_results = {l: {m: None for m in metrics} for l in labels}
test_results = {l: {m: None for m in metrics} for l in labels}
n_folds = 5

for label in labels:
    label_result_path = os.path.join(results_path, label)

    label_cv_results = {m: [] for m in metrics}

    for i in range(1, n_folds + 1):
        cv_fold_path = os.path.join(label_result_path, f"{i}.csv")
        df = pd.read_csv(cv_fold_path)
        final_row = df.iloc[-1].to_dict()

        for m in metrics:
            label_cv_results[m].append(final_row[m])

    test_path = os.path.join(label_result_path, "test.csv")
    df = pd.read_csv(cv_fold_path)
    final_row = df.iloc[-1].to_dict()

    for m in metrics:
        test_results[label][m] = final_row[m]

    for m in metrics:
        mean = float(np.mean(label_cv_results[m]))
        std = float(np.std(label_cv_results[m]))
        ci = std / (n_folds**0.5) * 1.96
        cv_results[label][m] = (mean, std, ci)

In [ ]:
for label in labels:
    print(label)
    for m in metrics:
        mean, std, ci = cv_results[label][m]
        print(f"{m}: {mean:.4f} ± {ci:.4f}")
    print()

In [ ]:
for label in labels:
    print(label)
    for m in metrics:
        mean = test_results[label][m]
        print(f"{m}: {mean:.4f}")
    print()